In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, BulkIndexError

In [2]:
# 데이터 읽기
holidays_path = r"/home/user1/project/shop_Modeling/data/e-commerce_data/holidays.csv"
holidays = pd.read_csv(holidays_path)

In [3]:
# 데이터프레임의 크기 확인
print("데이터프레임의 크기:", holidays.shape)

데이터프레임의 크기: (48, 2)


In [4]:
holidays.head()

,date,holiday
0,2021-01-01,New Year
1,2021-01-07,Orthodox Christmas
2,2021-01-13,Old New Year
3,2021-01-19,Baptizing - Sacred Epiphany
4,2021-01-25,Students day


In [5]:
column_types = holidays.dtypes
print(column_types)

date       object
holiday    object
dtype: object


In [6]:
# # Elasticsearch 서버 연결
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [7]:
# Elasticsearch 서버에 ping 요청 보내기
if es.ping():
    print("Elasticsearch 서버에 성공적으로 연결되었습니다.")
else:
    print("Elasticsearch 서버에 연결하지 못했습니다.")

Elasticsearch 서버에 성공적으로 연결되었습니다.


In [8]:
# 인덱스가 이미 존재하는 경우 삭제
if es.indices.exists(index="e_holidays"):
    es.indices.delete(index="e_holidays")

In [9]:
# Elasticsearch에 색인
e_holidays = 'e_holidays'  # 원하는 색인명으로 변경

# Elasticsearch 인덱스 매핑 설정
mapping = {
    "mappings": {
        "properties": {
            "date" : {"type": "date", "format": "yyyy-MM-dd"},  # 날짜 필드,
            "holiday" : {"type": "keyword"}
        }
    }
}

In [10]:
# 인덱스 생성
es.indices.create(index="e_holidays", body=mapping, ignore=400)

/tmp/ipykernel_13063/3184820445.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="e_holidays", body=mapping, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'e_holidays'})

In [11]:
# Elasticsearch에 이미 색인된 데이터 삭제
es.indices.delete(index="e_holidays", ignore=[400, 404])

/tmp/ipykernel_13063/277080718.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="e_holidays", ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [12]:
# 데이터프레임에서 Elasticsearch에 색인할 문서 생성
documents = []

for _, row in holidays.iterrows():
    document = row.to_dict()

    # NaN이 아닌 경우에만 날짜를 ISO 형식으로 변환
    for key, value in document.items():
        if key == 'date' and pd.notna(value):
            document[key] = pd.to_datetime(value).isoformat()

    # NaN을 None으로 대체
    document = {key: (None if pd.isna(value) else value) for key, value in document.items()}

    documents.append({
        "_op_type": "index",
        "_index": e_holidays,
        "_source": document
    })

# Elasticsearch에 색인
try:
    success, failed = bulk(es, documents)
    print(f"Indexed {success} documents successfully.")
    if failed:
        for i, failure in enumerate(failed):
            print(f"Failed to index document {i + 1}: {failure}")
except BulkIndexError as e:
    print(f"Error indexing documents: {e}")

print("색인이 완료되었습니다.")

Indexed 48 documents successfully.
색인이 완료되었습니다.
